In [3]:
!wget "https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Pre-Processing/to-victim/raw%20pcap/CODE/smlddostrace.to-victim.20070804_134936.pcap"

--2024-06-29 12:25:04--  https://raw.githubusercontent.com/sudomaster00081/MscProj/main/Pre-Processing/to-victim/raw%20pcap/CODE/smlddostrace.to-victim.20070804_134936.pcap
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6541672 (6.2M) [application/octet-stream]
Saving to: ‘smlddostrace.to-victim.20070804_134936.pcap.1’

smlddostrace.to-vic 100%[===================>]   6.24M  --.-KB/s    in 0.1s    

2024-06-29 12:25:04 (49.8 MB/s) - ‘smlddostrace.to-victim.20070804_134936.pcap.1’ saved [6541672/6541672]



In [15]:

!pip install scapy minisom tqdm

In [16]:
import os
from tqdm import tqdm

In [17]:
import csv
from scapy.all import rdpcap, IP, TCP, UDP, ICMP
from collections import defaultdict
from datetime import datetime
import math

# def calculate_entropy(data):
#     value_counts = {}
#     for value in data:
#         value_counts[value] = value_counts.get(value, 0) + 1

#     entropy = 0
#     for count in value_counts.values():
#         probability = count / len(data)
#         entropy -= probability * math.log2(probability)

#     return entropy

# def process_pcap(pcap_file, flow_stats_file, traffic_data_file):
#     print(f"Processing {pcap_file}...")
#     packets = rdpcap(pcap_file)
#     print(f"Number of packets: {len(packets)}")
#     flow_stats = defaultdict(lambda: {'packet_count': 0, 'byte_count': 0, 'start_time': None, 'end_time': None})
#     traffic_data = []

#     with open(flow_stats_file, 'w', newline='') as f_stats, open(traffic_data_file, 'w', newline='') as f_traffic:
#         stats_writer = csv.writer(f_stats)
#         traffic_writer = csv.writer(f_traffic)

#         # Write headers
#         stats_writer.writerow(['timestamp', 'datapath_id', 'flow_id', 'ip_src', 'tp_src', 'ip_dst', 'tp_dst', 'ip_proto',
#                                'icmp_code', 'icmp_type', 'flow_duration_sec', 'flow_duration_nsec', 'idle_timeout',
#                                'hard_timeout', 'flags', 'packet_count', 'byte_count', 'packet_count_per_second',
#                                'packet_count_per_nsecond', 'byte_count_per_second', 'byte_count_per_nsecond'])
#         traffic_writer.writerow(['timestamp', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'protocol'])

#         for packet in packets:
#             if IP in packet:
#                 ip_src = packet[IP].src
#                 ip_dst = packet[IP].dst
#                 ip_proto = packet[IP].proto
#                 timestamp = float(packet.time)  # Convert EDecimal to float

#                 if TCP in packet:
#                     tp_src = packet[TCP].sport
#                     tp_dst = packet[TCP].dport
#                 elif UDP in packet:
#                     tp_src = packet[UDP].sport
#                     tp_dst = packet[UDP].dport
#                 elif ICMP in packet:
#                     tp_src = tp_dst = 0
#                     icmp_type = packet[ICMP].type
#                     icmp_code = packet[ICMP].code
#                 else:
#                     continue

#                 flow_id = f"{ip_src}{tp_src}{ip_dst}{tp_dst}{ip_proto}"

#                 # Update flow stats
#                 if flow_stats[flow_id]['start_time'] is None:
#                     flow_stats[flow_id]['start_time'] = timestamp
#                 flow_stats[flow_id]['end_time'] = timestamp
#                 flow_stats[flow_id]['packet_count'] += 1
#                 flow_stats[flow_id]['byte_count'] += len(packet)

#                 # Write to traffic_data.csv

#                 traffic_writer.writerow([datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S"),
#                                          ip_src, tp_src, ip_dst, tp_dst, ip_proto])

#         # Write flow stats
#         for flow_id, stats in flow_stats.items():
#             duration = stats['end_time'] - stats['start_time']
#             duration_sec = int(duration)
#             duration_nsec = int((duration - duration_sec) * 1e9)

#             packet_count_per_second = stats['packet_count'] / duration if duration > 0 else 0
#             packet_count_per_nsecond = stats['packet_count'] / (duration * 1e9) if duration > 0 else 0
#             byte_count_per_second = stats['byte_count'] / duration if duration > 0 else 0
#             byte_count_per_nsecond = stats['byte_count'] / (duration * 1e9) if duration > 0 else 0

#             stats_writer.writerow([
#                 datetime.fromtimestamp(stats['end_time']).strftime("%Y-%m-%d %H:%M:%S"),
#                 '1',  # datapath_id (assuming single switch)
#                 flow_id,
#                 ip_src,
#                 tp_src,
#                 ip_dst,
#                 tp_dst,
#                 ip_proto,
#                 icmp_code if 'icmp_code' in locals() else -1,
#                 icmp_type if 'icmp_type' in locals() else -1,
#                 duration_sec,
#                 duration_nsec,
#                 '0',  # idle_timeout
#                 '0',  # hard_timeout
#                 '0',  # flags
#                 stats['packet_count'],
#                 stats['byte_count'],
#                 packet_count_per_second,
#                 packet_count_per_nsecond,
#                 byte_count_per_second,
#                 byte_count_per_nsecond
#             ])

#     # Calculate and print entropy values
#     with open(traffic_data_file, 'r') as f:
#         reader = csv.reader(f)
#         next(reader)  # Skip header
#         src_ips = []
#         src_ports = []
#         dst_ports = []
#         protocols = []
#         for row in reader:
#             src_ips.append(row[1])
#             src_ports.append(int(row[2]))
#             dst_ports.append(int(row[4]))
#             protocols.append(int(row[5]))

#     print(f"Entropy of source IP address (etpSrcIP): {calculate_entropy(src_ips)}")
#     print(f"Entropy of source port (etpSrcP): {calculate_entropy(src_ports)}")
#     print(f"Entropy of destination port (etpDstP): {calculate_entropy(dst_ports)}")
#     print(f"Entropy of packet protocol (etpProtocol): {calculate_entropy(protocols)}")
#     print(f"Total number of packets: {len(src_ips)}")

# Usage
# pcap_file = 'smlddostrace.to-victim.20070804_134936.pcap'
# flow_stats_file = 'PredictFlowStatsfile.csv'
# traffic_data_file = 'traffic_data.csv'
# print("Gonna read")
# process_pcap(pcap_file, flow_stats_file, traffic_data_file)

In [7]:
import math
import csv
import time
from datetime import datetime
from collections import defaultdict
from scapy.all import rdpcap, IP, TCP, UDP, ICMP
from tqdm import tqdm

def calculate_entropy(data):
    value_counts = {}
    for value in data:
        value_counts[value] = value_counts.get(value, 0) + 1

    entropy = 0
    for count in value_counts.values():
        probability = count / len(data)
        entropy -= probability * math.log2(probability)

    return entropy

def process_pcap(pcap_file, flow_stats_file, traffic_data_file):
    print(f"Processing {pcap_file}...")
    packets = rdpcap(pcap_file)
    print(f"Number of packets: {len(packets)}")
    flow_stats = defaultdict(lambda: {'packet_count': 0, 'byte_count': 0, 'start_time': None, 'end_time': None})
    traffic_data = []

    with open(flow_stats_file, 'w', newline='') as f_stats, open(traffic_data_file, 'w', newline='') as f_traffic:
        stats_writer = csv.writer(f_stats)
        traffic_writer = csv.writer(f_traffic)

        # Write headers
        stats_writer.writerow(['timestamp', 'datapath_id', 'flow_id', 'ip_src', 'tp_src', 'ip_dst', 'tp_dst', 'ip_proto',
                               'icmp_code', 'icmp_type', 'flow_duration_sec', 'flow_duration_nsec', 'idle_timeout',
                               'hard_timeout', 'flags', 'packet_count', 'byte_count', 'packet_count_per_second',
                               'packet_count_per_nsecond', 'byte_count_per_second', 'byte_count_per_nsecond'])
        traffic_writer.writerow(['timestamp', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'protocol'])

        for packet in tqdm(packets, desc="Processing packets", ncols=100):
            if IP in packet:
                ip_src = packet[IP].src
                ip_dst = packet[IP].dst
                ip_proto = packet[IP].proto
                timestamp = float(packet.time)  # Convert EDecimal to float

                if TCP in packet:
                    tp_src = packet[TCP].sport
                    tp_dst = packet[TCP].dport
                elif UDP in packet:
                    tp_src = packet[UDP].sport
                    tp_dst = packet[UDP].dport
                elif ICMP in packet:
                    tp_src = tp_dst = 0
                    icmp_type = packet[ICMP].type
                    icmp_code = packet[ICMP].code
                else:
                    continue

                flow_id = f"{ip_src}{tp_src}{ip_dst}{tp_dst}{ip_proto}"

                # Update flow stats
                if flow_stats[flow_id]['start_time'] is None:
                    flow_stats[flow_id]['start_time'] = timestamp
                flow_stats[flow_id]['end_time'] = timestamp
                flow_stats[flow_id]['packet_count'] += 1
                flow_stats[flow_id]['byte_count'] += len(packet)

                # Write to traffic_data.csv
                traffic_writer.writerow([datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S"),
                                         ip_src, tp_src, ip_dst, tp_dst, ip_proto])

        # Write flow stats
        for flow_id, stats in flow_stats.items():
            duration = stats['end_time'] - stats['start_time']
            duration_sec = int(duration)
            duration_nsec = int((duration - duration_sec) * 1e9)

            packet_count_per_second = stats['packet_count'] / duration if duration > 0 else 0
            packet_count_per_nsecond = stats['packet_count'] / (duration * 1e9) if duration > 0 else 0
            byte_count_per_second = stats['byte_count'] / duration if duration > 0 else 0
            byte_count_per_nsecond = stats['byte_count'] / (duration * 1e9) if duration > 0 else 0

            stats_writer.writerow([
                datetime.fromtimestamp(stats['end_time']).strftime("%Y-%m-%d %H:%M:%S"),
                '1',  # datapath_id (assuming single switch)
                flow_id,
                ip_src,
                tp_src,
                ip_dst,
                tp_dst,
                ip_proto,
                icmp_code if 'icmp_code' in locals() else -1,
                icmp_type if 'icmp_type' in locals() else -1,
                duration_sec,
                duration_nsec,
                '0',  # idle_timeout
                '0',  # hard_timeout
                '0',  # flags
                stats['packet_count'],
                stats['byte_count'],
                packet_count_per_second,
                packet_count_per_nsecond,
                byte_count_per_second,
                byte_count_per_nsecond
            ])

    # Calculate and print entropy values
    with open(traffic_data_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        src_ips = []
        src_ports = []
        dst_ports = []
        protocols = []
        for row in reader:
            src_ips.append(row[1])
            src_ports.append(int(row[2]))
            dst_ports.append(int(row[4]))
            protocols.append(int(row[5]))

    print(f"\nEntropy of source IP address (etpSrcIP): {calculate_entropy(src_ips)}")
    print(f"Entropy of source port (etpSrcP): {calculate_entropy(src_ports)}")
    print(f"Entropy of destination port (etpDstP): {calculate_entropy(dst_ports)}")
    print(f"Entropy of packet protocol (etpProtocol): {calculate_entropy(protocols)}")
    print(f"Total number of packets: {len(src_ips)}")

In [8]:
def main():
    # Get all pcap files in current directory
    pcap_files = [file for file in os.listdir() if file.endswith('.pcap')]

    for pcap_file in tqdm(pcap_files, desc="Files", leave=True):
        # Generate output filenames based on original pcap filename
        base_name = os.path.splitext(pcap_file)[0]  # Get filename without extension
        flow_stats_file = f"{base_name}_FlowStatsfile.csv"
        traffic_data_file = f"{base_name}_traffic_data.csv"
        print(f"\nGonna read {pcap_file}")
        # Process each pcap file
        process_pcap(pcap_file, flow_stats_file, traffic_data_file)

In [10]:
main()

Files:   0%|          | 0/1 [00:00<?, ?it/s]


Gonna read smlddostrace.to-victim.20070804_134936.pcap
Processing smlddostrace.to-victim.20070804_134936.pcap...
Number of packets: 129333



Files: 100%|██████████| 1/1 [00:41<00:00, 41.08s/it]

Entropy of source IP address (etpSrcIP): 3.742273164234512
Entropy of source port (etpSrcP): 1.1844544919480435
Entropy of destination port (etpDstP): 3.853294495346996
Entropy of packet protocol (etpProtocol): 0.8569946350286056
Total number of packets: 129333


129333


TypeError: rdpcap() got an unexpected keyword argument 'verbose'

In [ ]:
import math
import csv
import time
from datetime import datetime
from collections import defaultdict
from scapy.all import rdpcap, IP, TCP, UDP, ICMP
from tqdm import tqdm

def calculate_entropy(data):
    value_counts = {}
    for value in data:
        value_counts[value] = value_counts.get(value, 0) + 1

    entropy = 0
    for count in value_counts.values():
        probability = count / len(data)
        entropy -= probability * math.log2(probability)

    return entropy

def process_pcap(pcap_file, flow_stats_file, traffic_data_file):
    print(f"Reading {pcap_file}...")
    packets = rdpcap(pcap_file)
    print(f"Number of packets: {len(packets)}")
    flow_stats = defaultdict(lambda: {'packet_count': 0, 'byte_count': 0, 'start_time': None, 'end_time': None})
    traffic_data = []

    with open(flow_stats_file, 'w', newline='') as f_stats, open(traffic_data_file, 'w', newline='') as f_traffic:
        stats_writer = csv.writer(f_stats)
        traffic_writer = csv.writer(f_traffic)

        # Write headers
        stats_writer.writerow(['timestamp', 'datapath_id', 'flow_id', 'ip_src', 'tp_src', 'ip_dst', 'tp_dst', 'ip_proto',
                               'icmp_code', 'icmp_type', 'flow_duration_sec', 'flow_duration_nsec', 'idle_timeout',
                               'hard_timeout', 'flags', 'packet_count', 'byte_count', 'packet_count_per_second',
                               'packet_count_per_nsecond', 'byte_count_per_second', 'byte_count_per_nsecond'])
        traffic_writer.writerow(['timestamp', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'protocol'])

        for packet in tqdm(packets, desc="Processing packets", ncols=100):
            if IP in packet:
                ip_src = packet[IP].src
                ip_dst = packet[IP].dst
                ip_proto = packet[IP].proto
                timestamp = float(packet.time)  # Convert EDecimal to float

                if TCP in packet:
                    tp_src = packet[TCP].sport
                    tp_dst = packet[TCP].dport
                elif UDP in packet:
                    tp_src = packet[UDP].sport
                    tp_dst = packet[UDP].dport
                elif ICMP in packet:
                    tp_src = tp_dst = 0
                    icmp_type = packet[ICMP].type
                    icmp_code = packet[ICMP].code
                else:
                    continue

                flow_id = f"{ip_src}{tp_src}{ip_dst}{tp_dst}{ip_proto}"

                # Update flow stats
                if flow_stats[flow_id]['start_time'] is None:
                    flow_stats[flow_id]['start_time'] = timestamp
                flow_stats[flow_id]['end_time'] = timestamp
                flow_stats[flow_id]['packet_count'] += 1
                flow_stats[flow_id]['byte_count'] += len(packet)

                # Write to traffic_data.csv
                traffic_writer.writerow([datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S"),
                                         ip_src, tp_src, ip_dst, tp_dst, ip_proto])

        # Write flow stats
        for flow_id, stats in flow_stats.items():
            duration = stats['end_time'] - stats['start_time']
            duration_sec = int(duration)
            duration_nsec = int((duration - duration_sec) * 1e9)

            packet_count_per_second = stats['packet_count'] / duration if duration > 0 else 0
            packet_count_per_nsecond = stats['packet_count'] / (duration * 1e9) if duration > 0 else 0
            byte_count_per_second = stats['byte_count'] / duration if duration > 0 else 0
            byte_count_per_nsecond = stats['byte_count'] / (duration * 1e9) if duration > 0 else 0

            stats_writer.writerow([
                datetime.fromtimestamp(stats['end_time']).strftime("%Y-%m-%d %H:%M:%S"),
                '1',  # datapath_id (assuming single switch)
                flow_id,
                ip_src,
                tp_src,
                ip_dst,
                tp_dst,
                ip_proto,
                icmp_code if 'icmp_code' in locals() else -1,
                icmp_type if 'icmp_type' in locals() else -1,
                duration_sec,
                duration_nsec,
                '0',  # idle_timeout
                '0',  # hard_timeout
                '0',  # flags
                stats['packet_count'],
                stats['byte_count'],
                packet_count_per_second,
                packet_count_per_nsecond,
                byte_count_per_second,
                byte_count_per_nsecond
            ])

    # Calculate and print entropy values
    with open(traffic_data_file, 'r') as f:
        reader = csv.reader(f)
        next(reader)  # Skip header
        src_ips = []
        src_ports = []
        dst_ports = []
        protocols = []
        for row in reader:
            src_ips.append(row[1])
            src_ports.append(int(row[2]))
            dst_ports.append(int(row[4]))
            protocols.append(int(row[5]))

    print(f"\nEntropy of source IP address (etpSrcIP): {calculate_entropy(src_ips)}")
    print(f"Entropy of source port (etpSrcP): {calculate_entropy(src_ports)}")
    print(f"Entropy of destination port (etpDstP): {calculate_entropy(dst_ports)}")
    print(f"Entropy of packet protocol (etpProtocol): {calculate_entropy(protocols)}")
    print(f"Total number of packets: {len(src_ips)}")

